## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучить общую информацию. 

In [60]:
import pandas as pd
data = pd.read_csv('data.csv')

In [61]:
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

In [62]:
print(data['days_employed'].isnull().sum())              

2174


In [63]:
# количество пропусков в трудовом стаже и доходе соответствуют разности между
# общим количеством строк и пустых значений в "доходном" и "трудовом" столбцах - 2174

len(data[(data['days_employed'].isna()) & (data['total_income'].isna())])

2174

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


### Вывод

1. В самом начале я открыл файл, чтобы посмотреть "глазами" на столбцы из таблицы и оценить общую структуру списков и количество пропущенных значений в определенных стролбцах (использовал print()/head() и info() соответственно). 
2. При просмотре обращает на себя внимание пропущенные ('NaN') значения в столбце days_employed. После дальнейшего осмотра таблицы, замечаю, что в тех же строках, что и в столбце days_employed, есть отсутсвующие значения в столбце total_income (строки 12,26,29). Соответственно, появились эти пропуски не случайно. Это просто разные характеристики одного блока данных о работе заёмщика. Убеждаюсь в этом при подсчете количества нулевых значений в двух столбцах. Потом еще разок соотносим общие данные по таблице и количество пропущенных значений в двух столбцах: каждый столбец содержит по 2174 пропущенных значений (21535 - 19351). 
3. Еще одна проблема, которая существуюет в столбце с количеством рабочих дней, это то, что многие из значений имеют отрицательные значения. Отрицательные трудовой стаж заработать проблематично, поэтому мы можем предположить, что скорее всего возникла какая-то проблема с выгрузкой данных и поэтому они приходят в таблицу с минусом перед их истинным значением. Тоже самое в трудовых днях- присутствуют аномально высокие значения. 
4. Для начала нужно "почистить" данные таблицы. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [65]:
#print(data.head())

# убираем отрицательные значения из колонок с рабочими днями и количеством детей
data['children'] = data['children'].abs()
data['days_employed'] = data['days_employed'].abs()

In [66]:
print(data['days_employed'].mean())
print(data['days_employed'].median())

66914.72890682195
2194.220566878695


In [67]:
print(data['total_income'].mean())
print(data['total_income'].median())

167422.3022081719
145017.93753253992


In [68]:
income_median = data['total_income'].median()

In [69]:
# замена отсутствующих значений в двух столбцах на нули и медианные значения соответственно

data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(income_median)

In [70]:
data.isnull().sum() 

# пустых значений больше нет

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [71]:
# убираем артефактное значение 20 в столбце с детьми

print(data['children'].value_counts())
data = data[data['children'] != 20]

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


### Вывод

1. Для избавления от отрицательных значений в столбцах с рабочим стажем и зарплатами используем метод abs()
2. Cреднее проработанных дней равняется 66914.72, а медиана - 2194.22. В силу того, что присутствует большое количество безработных, мы видим такие результаты. Судя по этим результатам, крайне высокие значения в данных cдвигают "хвост" нашей выборки вправо (отрицательная симметрия). Вместо отсутствующих значений, мы внесем ноль, чтобы потом было удобней совершать различные действие с столбцом трудового стажа. Мы не будем использовать медиану или среднее, так как отсутствие трудового стажа дает важную информацию касательно заёмщиков, и соответственно ее лучше оставить.  
3. Аналогичная ситуация сложилась со значениями в столбце total_income: среднее - 167422.3, медиана - 145017.93. Но в столбце с зарплатами, разбежка не такая огромная как в стажевом столбце. Поэтому мы можем заключить, что использование медианы для замены отсутствующих значений более целесообразно.


### Замена типа данных

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21449 non-null  int64  
 1   days_employed     21449 non-null  float64
 2   dob_years         21449 non-null  int64  
 3   education         21449 non-null  object 
 4   education_id      21449 non-null  int64  
 5   family_status     21449 non-null  object 
 6   family_status_id  21449 non-null  int64  
 7   gender            21449 non-null  object 
 8   income_type       21449 non-null  object 
 9   debt              21449 non-null  int64  
 10  total_income      21449 non-null  float64
 11  purpose           21449 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


In [73]:
data['children'] = pd.to_numeric(data['children'], errors = 'coerce')

In [74]:
data['days_employed'] = data['days_employed'].astype('int')

### Вывод

Тип данных столбца с трудовым стажем в днях изменен с float64 на int64, чтобы данные в нём соответствовали логике того, что они отражают(целые дни). Т.к изначально данные были не строковые, то метод to_numeric() не подходил. 
Поэтому я использовал метод astype() для перевода в нужный мне тип данных.
Данные в столбце с детьми тоже были переведены в целочисленный тип integer (т.к 2.5 ребенка не есть хорошо).

### Обработка дубликатов

In [75]:
# приводим все текстовые значения в образовании к общему виду

data['education'] = data['education'].str.lower()



In [76]:
# избавляемся от дупликатов

data = data.drop_duplicates().reset_index(drop=True)



In [77]:
print(data.duplicated().sum())

0


### Вывод

1. В столбце с образованием текст был разного размера. Чтобы привести все значения этого столбца к общему виду, я использовал метод str.lower() 
2. Все дупликаты, которые были обмнаружены при совместном применении duplicated() и sum(), был удалены 

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()

# функция для разбития строк на леммы и дальнейшего создания отдельного столбца где эти леммы будут храниться

def purpose_function(row):
    text = row['purpose']
    lemma = ''.join(m.lemmatize(text))
    lemma = lemma.replace("\n","")
    return lemma

data['updated_purpose'] = data.apply(purpose_function, axis = 1)

# счётчик для подсчёта наиболее расространённых лемм

from collections import Counter
Counter(data['updated_purpose'])

# создание функции для выделения отдельных категорий наиболее распространённых лем
# и столбца где эти категории будут храниться

def purpose_category(row):
    if 'автомобиль' in row['updated_purpose']:
        return 'автомобиль'
    if 'жилье' in row['updated_purpose']:
        return 'жильё'
    if 'свадьба' in row['updated_purpose']:
        return 'свадьба'
    if 'недвижимость' in row['updated_purpose']:
        return 'недвижимость'
    if 'образование' in row['updated_purpose']:
        return "образование"

data['purpose_group'] = data.apply(purpose_category, axis =1)



### Вывод

1. После лемматизации всех значений в столбце purpose, мы создаем отдельный столбец, где хранятся полученные лемы.
При помощи счетчика Counter подсчитываем леммы и выделяем пять категорий: авто, жилье, недвижимость, свадьба, образование
2. Пишем функцию для категоризации лемм в пять вышеуказанных категорий. Скалдываем наши категории в один столбец.
(не писал данную функцию в следующем модуле, т.к здесь она идет по логике)

In [14]:
Counter(data['updated_purpose'])

Counter({'покупка жилье': 643,
         'приобретение автомобиль': 461,
         'дополнительный образование': 457,
         'сыграть свадьба': 761,
         'операция с жилье': 648,
         'образование': 445,
         'на проведение свадьба': 764,
         'покупка жилье для семья': 637,
         'покупка недвижимость': 619,
         'покупка коммерческий недвижимость': 658,
         'покупка жилой недвижимость': 604,
         'строительство собственный недвижимость': 629,
         'недвижимость': 632,
         'строительство недвижимость': 619,
         'на покупка подержать автомобиль': 472,
         'на покупка свой автомобиль': 505,
         'операция с коммерческий недвижимость': 646,
         'строительство жилой недвижимость': 623,
         'жилье': 642,
         'операция со свой недвижимость': 626,
         'автомобиль': 970,
         'заниматься образование': 408,
         'сделка с подержанный автомобиль': 482,
         'получение образование': 441,
         'свадьба': 79

### Категоризация данных

In [78]:
#создание функции для категоризации по возрасту

def age_group_funct(dob_years):
    if dob_years < 30:
        return 'young'
    if dob_years <= 45:
        return 'middle_aged'
    if dob_years <= 60:
        return 'mature'
    if dob_years <= 80:
        return 'seniors'
    else:
        return 'elderly'

data['age_group'] = data['dob_years'].apply(age_group_funct) 
    
data.groupby('age_group')['days_employed'].mean()


age_group
mature          96600.768143
middle_aged      5324.526544
seniors        261940.822337
young            3765.770101
Name: days_employed, dtype: float64

In [79]:
#создание функции для категоризации по наличию/отсутствию детей

def kids_function(children):
    if children < 1 :
        return 'childless'
    if children < 2:
        return 'one_kid'
    if children < 3:
        return '2_kids'
    else:
        return 'many_kids'
    
data['children_group'] = data['children'].apply(kids_function)


In [80]:
# создание функции для категоризации по доходу

data['total_income'].quantile([.25, .5, .75])

def income_function(income):
    if income < 107528.875:
        return 'low_income'
    if income < 145017.938:
        return 'middle_income'
    if income < 195784.746:
        return 'high_income'
    else:
        return 'very_high_income'

data['income_group'] = data['total_income'].apply(income_function)    

### Вывод

1. Написанные функции позволяют категоризировать данные по наличи детей, возрасту и доходу.
Я выделил возрастные группы опираясь на то, как их часто выделяют в различных анкетах или демографических шкалах.
При выделении групп с детьми я обратился к некоторым источникам и использовал общую логику.
Для выделения различных групп в доходе я использовал квартили, которые разбивают все значения столбца с доходом на четыре
равные части. Таким образом у нас есть некоторое общее представление о доходе заёмщиков.

2. После написания каждой из функций я применял ее к целевому столбцу и на основе полученных операций сохранял результаты в отдельном столбце. Таким образом мы получили новые столбцы age_group, children_group, income_group + purpose_group.  

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [81]:
data['debt'].value_counts()

0    19645
1     1733
Name: debt, dtype: int64

In [82]:
indebted_percent = ((1733 / 19645) * 100)

In [83]:
print(indebted_percent)

8.821583100025451


In [84]:
data['children'].value_counts()



0    14091
1     4855
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [85]:
childfree_percent = ((14091 / 21378) *100)
print(childfree_percent)

65.91355599214145


### Вывод

Существует зависимость между наличием детей выплатой кредита 

1. Процент имеющих задолженности по долгам составляет 8.8% от всех заёмщиков. 
2. Процент неимеющих детей заемщиков составляет 65%.
3. При простом подсчете количества детей у задолжавших банку заемщиках при помощи сводной таблицы, мы видим, что бездетные лидируют с большим отрывом.
Но в тоже время бездетные в принципе составляют большую (65%) часть заемщиков. Поэтому нам нужно расчитать среднее из столбца с детьми, чтобы иметь пропорцию заемщиков с разным количеством детей по отношению к остальным.
4. После создания сводной таблицы по интересующим на параметрам, мы можем заметить, что результаты сильно поменялись. 
В среднем по всей выборке, чуть больше 8% заемщиков имеют долги перед банком. В то, время как заемщики с четырьмя, двумя и одним ребенком на 15%, 14% и 11% более склонны иметь задолженность по сравнению со средним по выборке. В тоже время заемщики с отсутствием детей на 9% меньше склонны иметь задолженности по отношению к среднему выборки, а с 3 детьми имеют показатели идентичные среднему по выборке.
Соответственно, заемщики с 4, 2, и 1 одним ребенком являются наиболее вероятными должниками. Бездетные возвращают кредит лучше всего.


In [86]:
result = (data.groupby('children_group')
              .agg({'children':'count', 'debt':'sum'})
              .rename(columns={'children':'total'})
              .reset_index()
         )

result['debt_ratio'] = result['debt'] / result['total'] * 100
result.round(2).sort_values('debt_ratio')

# соотношение показателей разных групп c детьми со средним по выборке

#((0.098(0.095, 0.092) / 0.081) * 100) - 100) 

,children_group,total,debt,debt_ratio
1,childless,14091,1063,7.54
2,many_kids,380,31,8.16
3,one_kid,4855,445,9.17
0,2_kids,2052,194,9.45


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [87]:
print(data['family_status'].value_counts())

женат / замужем          12290
гражданский брак          4139
Не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64


In [88]:
# создание сводной таблицы  

data_pivot = data.pivot_table(index = ['family_status'], values ='debt', columns = 'gender', aggfunc = 'mean', margins = True)

print(data_pivot)




gender                        F         M  XNA       All
family_status                                           
Не женат / не замужем  0.068406  0.144052  NaN  0.097465
в разводе              0.065241  0.089147  NaN  0.070411
вдовец / вдова         0.057778  0.200000  NaN  0.065969
гражданский брак       0.081777  0.117602  0.0  0.093018
женат / замужем        0.068176  0.087939  NaN  0.075509
All                    0.070079  0.102483  0.0  0.081065


### Вывод

Семейный статус влияет на возврат долга банку заёмщиком

1. Аналогично процедурам в предущем модуле я рассчитываю аналогичные показатели для категории семейный статус.
2. Не женатые на 19% более склонны иметь задолженность по сравнени со средним по выборке, а заёмщики в гражданском браке на 14%.
3. В тоже время вдовы/вдовцы наиболее надежные заемщики и в среднем на 20% менее вероятно, что они будут иметь задолженность по сравнению со средним по выборке. Заемщики со статусом "в разводе" на 14 % менее склонны и иметь задолженность по сравнению со средним по выборке. Женатые/замужние тоже надежней средней по выборке на 8%.
4. Таким образом, не женатые и живущие гражданским браком являются наимее надежными заёмщиками в категории "семейный статус". В тоже время вдовы и вдовцы являются наиболее надежными. По надежности вслед за ними следуют разведенные, а потм и женатые/замужние.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

### Вывод

Уровень дохода имеет влияние на вероятность выплаты долга заёмщиками

1. Наиболее являются заёмщики с наиболее высоким доходом. Они на 13% менее склонны не возвращать кредит по сравнению со средним по выборке. 
2. В тоже время заёмщики из второго по величине квартиля по доходам являются наиболее частыми заёмщиками с долгом - на 8 процентом более склонны не платить по долгам. 
3. Заемщики с низким и средним уровнем дохода близки по своим значениям к среднему по выборке.


- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot = data.pivot_table(index = ['purpose_group'], values ='debt', columns = 'gender', aggfunc = 'mean', margins = True)

### Вывод

Цель, на которую берется кредит имеет влияние на его возврат.

1. Заёмщики кредита на автомобиль и образование более склонны к нарушению долговых обязательств по сравнению со средним по выборке (на 14% и 13% соответственно). 
2. В тоже время, занимающие деньги на жилье, наоборот, на 15 % менее склонны к невозврату кредиту в срок по сравнению со средним по выборке. Берущие деньги на недвижимость на 9% менее склонны не вернуть деньги. Занимающие деньги на свадьбу очень близки к показателям средней по выборке.


### Шаг 4. Общий вывод

Мы можем заключить, что в выделенные факторы имееют определенное влияние на вероятность выплаты долга:
1. Заёмщики без детей являются наиболее надеждными, по стравнению с теми, кто имеет 1,2,4 ребенка. 
2. Овдовевшие, разведенные и женатые являются более надежными заемщиками по сравнению с незамужними и живущими гражданским браком кредитополучателсями. 
3. Неожиданно я нашел, что заёмщики из второго по величине квартиля более склонны к наличию задолженности, чем те, что имеют средний или низкий доход.
4. Заёмщики, берущие кредит с целью покупки автомобиля или образования более склонны к невыплате, чем те, что берут кредит на жилье, недвижимость или свадьбу.